In [1]:
## getting all company names

import os
from bs4 import BeautifulSoup

# Directory containing the batch files
directory = 'batches'

# List to store all company names
all_company_names = []

# Iterate over all files in the directory
for filename in os.listdir(directory):
    #print(f"Processing file: {filename}")
    #if filename.endswith('.html') or filename == 'all':  # Adjust the condition based on your file types
    filepath = os.path.join(directory, filename)
    
    # Load the HTML content from the file
    with open(filepath, 'r') as file:
        html_content = file.read()
    
    # Print the first 500 characters of the HTML content for debugging
    #print(f"HTML content of {filename} (first 500 chars): {html_content[:500]}")
    
    # Parse the HTML using Beautiful Soup
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Find all anchor tags with class '_company_99gj3_339'
    company_links = soup.find_all('a', class_='_company_99gj3_339')
    #print(f"Found {len(company_links)} company links in {filename}")
    
    # Extract the company names from the nested spans
    company_names = [link.find(class_='_coName_99gj3_454').text for link in company_links if link.find(class_='_coName_99gj3_454')]
    
    # Add the company names to the list
    all_company_names.extend(company_names)

# Print or process the company names
#print(f"Total company names found: {len(all_company_names)}")
print(all_company_names)

['Reddit', 'Kiko', 'Clickfacts', 'TextPayMe', 'Loopt', 'Infogami', 'Memamp', 'Simmery', 'Whatnot', 'Jet Admin', 'Ochre Bio', 'Ansa Biotechnologies', 'Yassir', 'Jenfi', 'Legionfarm', 'Menten AI', 'Belvo', 'Equator Therapeutics', 'Dapi', 'Gas Pos', 'Humanly', 'Prelay', 'WorkPay', 'Airbyte', 'Tambua Health', 'Living Carbon', 'PowerX', 'SEND', '99minutos', 'Atmo', 'borong', 'Recall.ai', 'Healthlane', 'Stable', 'Sakneen', 'GiveAway', 'GuruHotel', 'Nomic Bio', 'Ramani', 'Handl', 'Zynq', 'STARK BANK', 'Termii', 'BeGo', 'Linkana', 'Fondeadora', 'Tajir', 'Moons', 'Bamboo', 'Hent', 'ElectroNeek', 'Swipe - (Float, Accounteer)', 'SOMATIC', 'Avion', 'Daedalus', 'Oda', 'Nexu', 'Isabl', 'OpenBiome', 'Breathe Well-being', 'Angle Health', 'Raycast', 'Easyplan', 'Pahamify', 'HireSweet', 'Spenny', 'EduRev', 'Bikry', 'Hiration', 'Zeo Route Planner', 'Morphle Labs', 'TagMango', 'PillarPlus', 'farmtheory', 'UrbanKisaan', 'Flasho', 'StayQrious', 'Able', 'Kosh', 'DeepSource', 'Upflow', 'PostEra', 'Skypher', '

In [2]:
## getting all company links

#need to get all the links
import os
from bs4 import BeautifulSoup

# Directory containing the batch files
directory = 'batches'

# List to store all company links
all_company_links = []

# Iterate over all files in the directory
for filename in os.listdir(directory):
    #print(f"Processing file: {filename}")
    filepath = os.path.join(directory, filename)
    
    # Load the HTML content from the file
    with open(filepath, 'r') as file:
        html_content = file.read()
    
    # Print the first 500 characters of the HTML content for debugging
    #print(f"HTML content of {filename} (first 500 chars): {html_content[:500]}")
    
    # Parse the HTML using Beautiful Soup
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Find all anchor tags with class '_company_99gj3_339'
    company_links = soup.find_all('a', class_='_company_99gj3_339')
    #print(f"Found {len(company_links)} company links in {filename}")
    
    # Extract the href attribute to get the company links
    links = [link.get('href') for link in company_links]
    
    # Add the company links to the list
    all_company_links.extend(links)

# Print or process the company links
print(f"Total company links found: {len(all_company_links)}")

Total company links found: 4688


In [26]:
#what can i do next
#iterate over all links and get data
"""
data i need to aquire 

- name
- all tags
- description and active founder description
- url to page
- founded
- team size
- location
- socials
- all founder names and roles and socials
"""
import asyncio
from playwright.async_api import async_playwright

async def run(playwright):
    browser = await playwright.chromium.launch(headless=False)  # Set headless=False if you want to see the browser
    page = await browser.new_page()

    # List of company URLs
    company_urls = [
        "https://www.ycombinator.com/companies/backtype",
        "https://www.ycombinator.com/companies/deel",
        "https://www.ycombinator.com/companies/poll-everywhere",
        "https://www.ycombinator.com/companies/flagr",
        "https://www.ycombinator.com/companies/parcelbio", 
        # Add more URLs here
    ]

    all_company_data = []

    for url in company_urls:
        print(f"Navigating to {url}")
        await page.goto(url)
        print(f"Successfully navigated to {url}")

        company_data = []

        # Get name
        print("Extracting company name...")
        company_name = await page.text_content('h1.font-extralight')
        company_name = company_name.strip() if company_name else "Name not found"
        company_data.append(company_name)
        print(f"Company name: {company_name}")

        # Check if active
        print("Checking if company is active...")
        active_div_selector = 'div.flex.flex-row.items-center.justify-between > div.mr-\[6px\].h-3.w-3.rounded-full.bg-green-500'
        active_div = await page.query_selector(active_div_selector)
        is_active = "Active" if active_div else "Not Active"
        company_data.append(is_active)
        print(f"Company is {is_active}")

        # Get profile picture
        print("Extracting profile picture...")
        try:
            profile_picture = await page.wait_for_selector('div.h-32.w-32.shrink-0.clip-circle-32 > img', state='attached', timeout=100)
            profile_picture_url = await profile_picture.get_attribute('src') if profile_picture else "Not available"
        except Exception as e:
            print(f"Failed to find profile picture: {str(e)}")
            profile_picture_url = "Not available"
        company_data.append(profile_picture_url)
        print(f"Profile picture URL: {profile_picture_url}")

        # Extract all tags
        print("Extracting tags...")
        tags_elements = await page.query_selector_all('div.align-center.flex.flex-row.flex-wrap.gap-x-2.gap-y-2 > a')
        tags = [await tag.text_content() for tag in tags_elements if await tag.text_content()]
        company_data.append(tags)
        print(f"Tags: {tags}")

        # Get description
        print("Extracting description...")
        description = await page.text_content('p.whitespace-pre-line')
        description = description.strip() if description else "Description not available"
        company_data.append(description)
        print(f"Description: {description}")

        # Extract foundational details, team size, and location
        print("Extracting foundational details, team size, and location...")
        founded_selector = 'div.ycdc-card > div.space-y-0\\.5 > div:nth-child(1) > span:last-child'
        founded = await page.text_content(founded_selector)
        team_size_selector = 'div.ycdc-card > div.space-y-0\\.5 > div:nth-child(2) > span:last-child'
        team_size = await page.text_content(team_size_selector)
        location_selector = 'div.ycdc-card > div.space-y-0\\.5 > div:nth-child(3) > span:last-child'
        location = await page.text_content(location_selector)
        company_data.append(founded)
        company_data.append(team_size)
        company_data.append(location)
        print(f"Founded: {founded}, Team Size: {team_size}, Location: {location}")

        # Get socials
        print("Extracting social media links...")
        socials = {}
        social_media_types = {
            "LinkedIn": 'a[aria-label="LinkedIn profile"]',
            "Twitter": 'a[aria-label="Twitter account"]',
            "Facebook": 'a[aria-label="Facebook profile"]',
            "Crunchbase": 'a[aria-label="Crunchbase profile"]',
            "GitHub": 'a[aria-label="Github profile"]'
        }
        for media, selector in social_media_types.items():
            try:
                href = await page.wait_for_selector(selector, state='attached', timeout=100)
                if href:
                    socials[media] = await href.get_attribute('href')
                else:
                    socials[media] = "Not available"
            except Exception as e:
                print(f"Failed to find {media} link: {str(e)}")
                socials[media] = "Not available"
        company_data.append(socials)
        print(f"Social media links: {socials}")

        # Extract founders' data
        print("Extracting founders' data...")
        founders_info = await extract_founder_info(page)
        company_data.append(founders_info)
        print(f"Founders Information: {founders_info}")

        all_company_data.append(company_data)
        print(f"Company data: {company_data}")

        await asyncio.sleep(100)  # Wait for a bit (1 second)
        await page.wait_for_timeout(3000)  # Wait for 3000 milliseconds (3 seconds)
        print(f"Visited {url}")

    await browser.close()

async def extract_founder_info(page):
    # Selector for the container that holds all founders' information
    founder_elements_selector = 'div.flex.flex-row.items-center.gap-x-3'
    # Get all founder elements
    founder_elements = await page.query_selector_all(founder_elements_selector)
    print("Founders elements:", founder_elements)
    all_founders_data = []
    for founder in founder_elements:
        # Extract the founder's name
        name = await founder.query_selector('div.font-bold')
        name_text = await name.text_content() if name else "Name not found"
        # Extract social media links
        socials = {}
        social_media_types = {
            "LinkedIn": 'a[aria-label="LinkedIn profile"]',
            "Twitter": 'a[aria-label="Twitter account"]'
        }
        for media, selector in social_media_types.items():
            social_link = await founder.query_selector(selector)
            social_href = await social_link.get_attribute('href') if social_link else "Not available"
            socials[media] = social_href
        # Compile all data for this founder
        founder_data = {
            "name": name_text.strip(),
            "socials": socials
        }
        all_founders_data.append(founder_data)
    return all_founders_data

async def main():
    async with async_playwright() as playwright:
        await run(playwright)

# Check if the current event loop is running and use it
loop = asyncio.get_event_loop()
if loop.is_running():
    task = loop.create_task(main())
else:
    asyncio.run(main())

Navigating to https://www.ycombinator.com/companies/backtype
Successfully navigated to https://www.ycombinator.com/companies/backtype
Extracting company name...
Company name: BackType
Checking if company is active...
Company is Not Active
Extracting profile picture...
Profile picture URL: https://bookface-images.s3.amazonaws.com/small_logos/0ec00079f2609a11308b0e89a7e2ed64081516db.png
Extracting tags...
Tags: ['Y Combinator LogoS08', 'saas', 'data-engineering', 'San Francisco']
Extracting description...
Description: Description not available
Extracting foundational details, team size, and location...
Founded: 2008, Team Size: 0, Location: San Francisco
Extracting social media links...
Failed to find Facebook link: Page.wait_for_selector: Timeout 100ms exceeded.
Call log:
waiting for locator("a[aria-label=\"Facebook profile\"]")

Failed to find GitHub link: Page.wait_for_selector: Timeout 100ms exceeded.
Call log:
waiting for locator("a[aria-label=\"Github profile\"]")

Social media lin